In [1]:
import os
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.documents import Document
import sqlite3

In [2]:
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/tmp/ipykernel_12486/1698008280.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
/home/alee/.local/share/virtualenvs/chronosAI-bGPFpNdy/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
try:
    file_path = 'data/politicaNominas.md'
    print(f"El directorio de trabajo actual es: {os.getcwd()}")

    with open(file_path, "r", encoding="utf-8") as archivo:
        contenido = archivo.read()
        print(f"Se leyó el archivo '{file_path}'.")

        headersToSplitOn = [("#", "Header"), ("##", "Title")]

        markdown_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=headersToSplitOn
        )
        md_header_splits = markdown_splitter.split_text(contenido)

        for document in md_header_splits:
            lista = []

            # Extraer y mostrar los metadatos
            metadata = document.metadata
            page_content = document.page_content
            for key, value in metadata.items():
                lista.append(f"{value}{page_content}")
                print(
                    "##########################################################################"
                )
                print(f"{value}{page_content}")

        vector_store = Chroma.from_documents(
            md_header_splits,
            embeddings,
            collection_metadata={"hnsw:space": "cosine"},
            persist_directory="data/stores/nominas",
        )

except FileNotFoundError:
    print(f"El archivo '{file_path}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")

El directorio de trabajo actual es: /home/alee/Documents/MIA/chronosAI
Se leyó el archivo 'data/politicaNominas.md'.


In [4]:
model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [5]:
DATABASE_PATH = 'data/empleados.db'
TABLE_NAME = 'empleados'
COLUMNS_TO_EXTRACT = '*'

try:
    print(f"El directorio de trabajo actual es: {os.getcwd()}")

    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    print(f"Conexión exitosa a la base de datos: {DATABASE_PATH}")

    select_columns = ", ".join(COLUMNS_TO_EXTRACT)
    query = f"SELECT {select_columns} FROM {TABLE_NAME}"
    cursor.execute(query)
    data = cursor.fetchall()

    documents = []
    for row in data:
        content = ", ".join([f"{COLUMNS_TO_EXTRACT[i]}: {row[i]}" for i in range(len(COLUMNS_TO_EXTRACT))])
        metadata = dict(zip(COLUMNS_TO_EXTRACT, row))
        # Crea un objeto Document en lugar de un diccionario
        documents.append(Document(page_content=content, metadata=metadata))

    conn.close()
    print(f"Se extrajeron {len(documents)} documentos de la tabla '{TABLE_NAME}'.")

    # Crear el vector store ahora con la lista de objetos Document
    vector_store = Chroma.from_documents(
        documents,
        embeddings,
        collection_metadata={"hnsw:space": "cosine"},
        persist_directory="data/stores/empleados",
    )
    print(f"Vector store creado en: data/stores/empleados")

except sqlite3.Error as e:
    print(f"Error al interactuar con la base de datos: {e}")
except Exception as e:
    print(f"Ocurrió un error: {e}")

El directorio de trabajo actual es: /home/alee/Documents/MIA/chronosAI
Conexión exitosa a la base de datos: data/empleados.db
Se extrajeron 900 documentos de la tabla 'empleados'.
Vector store creado en: data/stores/empleados
